In [1]:
import pandas as pd
import requests

In [2]:
################################
# ETF 종목 코드가 없어서 사용 안함
################################
if False:
    code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 

    # 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
    # code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 

    # 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외. 
    code_df = code_df[['회사명', '종목코드']] 

    # 한글로된 컬럼명을 영어로 바꿔준다. 
    code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 
    code_df.head()

    print(code_df.info())
    print(code_df.loc[code_df['name'] == '삼성전자'])

In [4]:
# 웹에서 얻어올지 지정 
CSV_FILE:str = './data/233740_2022.09.05_2015.12.17.csv' # 없을 땐 None
    
MAX_SEARCH_PAGE:int = 168
STOCK_CODE:str = '233740'

In [5]:
# 종목 별 데이터를 네이버 Finanace에서 가져옴
# 컬럼: 날짜,종가,전일비,시가,고가,저가,거래량
def get_stock_data(code:str, max_search_page:int) -> pd.DataFrame:

    import requests 
    FAKE_HEADER = {'User-agent': 'Mozilla/5.0'}
    # KODEX 코스닥150 레버리지 일자데이터 url 가져오기 
    url = f'http://finance.naver.com/item/sise_day.nhn?code={code}'

    max_search_page = max(2, max_search_page) # 최소 1페이지는 검색 
    
    # 일자 데이터를 담을 df라는 DataFrame 정의 
    df_res = pd.DataFrame() 
    # 마자막 ROW를 찾기 위한 변수  
    prev_row_size = float('inf')

    # 1페이지에서 MAX_SEARCH_PAGE 페이지의 데이터만 가져오기 
    for page in range(1, MAX_SEARCH_PAGE): 
        pg_url = '{url}&page={page}'.format(url=url, page=page) 

        # header = 0 가져온 table의 첫 행을 DataFrame의 컬럼으로 설정
        # [0] 지정한 URL의 첫번째 table을 DataFrame으로 반환
        next_df = pd.read_html(requests.get(pg_url, headers=FAKE_HEADER).text, header=0)[0]
        next_df = next_df.dropna() 
        df_res = pd.concat([df_res, next_df])

        # 마지막 row 체크 
        if prev_row_size != float('inf') and prev_row_size != next_df.shape[0]: 
            break
        prev_row_size = next_df.shape[0]

    # 필요 없음 
    #df_res = df_res[["날짜","종가", "전일비", "시가", "고가", "저가", "거래량"]]

    # df.dropna()를 이용해 결측값 있는 행 제거 
    return df_res.dropna() 

In [6]:
# csv 파일 저장 (이름 규칙: code _최근날짜_가장예전날짜.csv)
def save_stock_csv(code_s:str, df_s: pd.DataFrame, data_dir:str = './data') -> None:
    print(df_s.iloc[0]['날짜'])
    print(df_s.iloc[-1]['날짜'])

    recent_date = df_s.iloc[0]['날짜']
    oldest_date = df_s.iloc[-1]['날짜']

    filename = f'{code_s}_{recent_date}_{oldest_date}'
    print(filename)
    
    df_s.to_csv(data_dir+'/'+filename+'.csv')

In [7]:
df_kodaq:pd.DataFrame = None 
    
if not CSV_FILE:
    print('Web Crawling!!!')
    df_kodaq = get_stock_data(STOCK_CODE, MAX_SEARCH_PAGE)

    # 상위 5개 데이터 확인하기 
    df_kodaq[:5]
    
    save_stock_csv(STOCK_CODE, df_kodaq)
else:
    print('From CSV file: ', CSV_FILE)
    df_kodaq = pd.read_csv(CSV_FILE)

From CSV file:  ./data/233740_2022.09.05_2015.12.17.csv


In [8]:
df_kodaq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1651 entries, 0 to 1650
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1651 non-null   int64  
 1   날짜          1651 non-null   object 
 2   종가          1651 non-null   float64
 3   전일비         1651 non-null   float64
 4   시가          1651 non-null   float64
 5   고가          1651 non-null   float64
 6   저가          1651 non-null   float64
 7   거래량         1651 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 103.3+ KB
